## Converting from Codepoint to Char and Back

In [ ]:
# wrong way
[char]103
[char]0x1f468
'was it too big?'
0x1f468 -gt [char]::MaxValue

g
InvalidArgument: 
Line |
   3 |  [char]0x1f468
     |  ~~~~~~~~~~~~~
     | Cannot convert value "128104" to type "System.Char". Error: "Value was either too large or too small for a character."
was it too big?
True


In [ ]:
# Codepoint (int32) to char/string
[char]::ConvertFromUtf32( 0x1f468 )

# codepoint from [char] to codepoint
$charList = '👨'.ToCharArray()
[char]::ConvertToUtf32( $charList[0], $charList[1] )

# it outputs in decimal, it's the same number
128104 -eq 0x1f468


👨
128104
True


Note: The return type of `[char]::ConvertFromUtf32` is actually a `[string]`, not a `[char]`

In [ ]:
function inspectString {
    param(
        [string]$InputText
    )
    $info = @{
        'Input' = $InputText
        'Utf16Length' = $InputText.Length
        'TotalCodepoints' = $inputText.EnumerateRunes().Value.count
        'Details' = $InputText.EnumerateRunes()
    }
    return [pscustomobject]$Info
    # return $Info
}

$str = '👨👨‍👩‍👧‍👦'
$strMan = '👨'
inspectString $str 
(inspectString $str ).Details | Ft -auto



Details          Input         Utf16Length TotalCodepoints
-------          -----         ----------- ---------------
{👨, 👨, ‍, 👩…} 👨👨‍👩‍👧‍👦          13               8


IsAscii IsBmp Plane Utf16SequenceLength Utf8SequenceLength  Value
------- ----- ----- ------------------- ------------------  -----
  False False     1                   2                  4 128104
  False False     1                   2                  4 128104
  False  True     0                   1                  3   8205
  False False     1                   2                  4 128105
  False  True     0                   1                  3   8205
  False False     1                   2                  4 128103
  False  True     0                   1                  3   8205
  False False     1                   2                  4 128102



In [ ]:
function inspectStrLength {
    
}

inspectStrLength $str
| Measure-Object -Property Utf16SequenceLength, Utf8SequenceLength -Sum | ft Sum, Property
$str.length




Sum Property
--- --------
 13 Utf16SequenceLength
 29 Utf8SequenceLength

13


In [ ]:
'other numbers'
[Text.Encoding]::UTF8.GetByteCount( $str )

[Text.Encoding]::Unicode.GetByteCount( $str )
[Text.Encoding]::Unicode.GetCharCount( $str )
[Text.Encoding]::Unicode.GetBytes( $str ).count

other numbers
29
26
MethodException: 
Line |
   5 |  [Text.Encoding]::Unicode.GetCharCount( $str )
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Cannot convert argument "bytes", with value: "👨👨‍👩‍👧‍👦", for "GetCharCount" to type "System.Byte[]": "Cannot convert value "👨👨‍👩‍👧‍👦" to type "System.Byte[]". Error: "Cannot convert value "👨👨‍👩‍👧‍👦" to type "System.Byte". Error: "Input string was not in a correct format."""
26


Converting a codepoint to a `[char]`, (actually a `[string]`)

Assume that Dotnet uses the encoding `UTF-16LE` by default, unless the docs explicitly say another. 
| Type | Description |
| - | -  |
| `[char]` | represesnts a single UTF16le `code-unit`<br>max value is **2-bytes** |
| `[String]` | a list of utf16le code-units | 
| `[Rune]` | represents a single Unicode Scalar Codepoint<br>in the range `[0..0x10ffff]`<br>Except the Surrogate range `[0xd800..0xdfff]`   |

docs:
- [Introduction to Character Encoding in `.NET`](https://docs.microsoft.com/en-us/dotnet/standard/base-types/character-encoding-introduction)
- [`[Char]`](https://docs.microsoft.com/en-us/dotnet/api/system.char?view=net-6.0)
- [`[String]`](https://docs.microsoft.com/en-us/dotnet/api/system.char?view=net-6.0)
- [`[Text.Rune]`](https://docs.microsoft.com/en-us/dotnet/api/system.text.rune?view=net-6.0)
- [`[Globalization.StringInfo]`](https://docs.microsoft.com/en-us/dotnet/api/system.globalization.stringinfo?view=net-6.0)

Note: **WindowsPowershell** does not have any `[Rune]` functions

See more:
- [Introduction to Character Encoding in `.NET`](https://docs.microsoft.com/en-us/dotnet/standard/base-types/character-encoding-introduction)
- [`System.String` and `char`](https://docs.microsoft.com/en-us/dotnet/api/system.string?view=net-6.0#char-objects-and-unicode-characters)